# Discrete Events

# ResumableFunctions

In [ ]:
using Pkg
pkg"add ResumableFunctions"
using ResumableFunctions

In [ ]:
@resumable function fibonnaci(n::Int64)
    a = zero(Int64)
    b = one(Int64)
    for i in 1:n
        @yield a
        a, b = b, a+b
    end
end

In [ ]:
for v in fibonnaci(16)
    println(v)
end

# Process-driven Discrete Event Simulation

In [ ]:
pkg"add SimJulia"
using SimJulia

## Basic Process

In [ ]:
@resumable function car(sim::Simulation)
    while true
        println(now(sim), ": I'm driving")
        @yield timeout(sim, 3)
        println(now(sim), ": I'm parked")
        @yield timeout(sim, 2)
    end
end

In [ ]:
sim = Simulation()
@process car(sim)
run(sim, 10)

## Waiting for a Process

In [ ]:
using Dates

In [ ]:
@resumable function chargingpoint(sim::Simulation, duration::Period)
    println(nowDatetime(sim), ": Charging station activated")
    @yield timeout(sim, duration)
    println(nowDatetime(sim), ": Charging station deactivated")
end

In [ ]:
@resumable function car(sim::Simulation)
    while true
        println(nowDatetime(sim), ": I'm driving")
        @yield timeout(sim, Hour(3))
        println(nowDatetime(sim), ": I'm parked")
        charging = @process chargingpoint(sim, Hour(2))
        @yield charging
    end
end       

In [ ]:
sim = Simulation(DateTime(2018, 10, 23, 8, 45, 0))
@process  car(sim)
run(sim, DateTime(2018, 10, 23, 24, 0, 0))

## Interrupting a Process

In [ ]:
@resumable function driver(sim::Simulation, charging::Process)
    busy_period = Minute(Int(floor(180*rand())))
    @yield timeout(sim, busy_period)
    if busy_period < Hour(2)
        println(nowDatetime(sim), ": I'm running out of time")
        @yield interrupt(charging)
    end
end

In [ ]:
@resumable function chargingpoint(sim::Simulation, duration::Period)
    println(nowDatetime(sim), ": Charging station activated")
    try
        @yield timeout(sim, duration)
    catch exc
        println(nowDatetime(sim), ": Charging interrupted")
    end
    println(nowDatetime(sim), ": Charging station deactivated")
end

In [ ]:
@resumable function car(sim::Simulation)
    while true
        println(nowDatetime(sim), ": I'm driving")
        @yield timeout(sim, Hour(3))
        println(nowDatetime(sim), ": I'm parked")
        charging = @process chargingpoint(sim, Hour(2))
        @yield @process driver(sim, charging)
    end
end

In [ ]:
sim = Simulation(DateTime(2018, 10, 23, 8, 45, 0))
@process  car(sim)
run(sim, DateTime(2018, 10, 23, 24, 0, 0))

# Resources

## Basic Queue

In [ ]:
@resumable function cargenerator(sim::Simulation, number::Int64, station::Resource)
    for i in 1:number
        @process car(sim, i, station)
    end
end

In [ ]:
@resumable function car(sim::Simulation, number::Int, station::Resource)
    while true
        println(nowDatetime(sim), ": ($number) I'm driving")
        @yield timeout(sim, Minute(Int(floor(180*rand()))))
        starttime = nowDatetime(sim)
        println(starttime, ": ($number) I'm parked")
        @yield request(station)
        stoptime = nowDatetime(sim)
        println(stoptime, ": ($number) I can charge after $(floor(stoptime - starttime, Minute)) minutes")
        @yield timeout(sim, Minute(Int(floor(120*rand()))))
        println(nowDatetime(sim), ": ($number) I'm charged'")
        @yield release(station)
    end
end        

In [ ]:
sim = Simulation(DateTime(2018, 10, 23, 8, 45, 0))
charging_station = Resource(sim, 2)
@process  cargenerator(sim, 8, charging_station)
run(sim, DateTime(2018, 10, 23, 24, 0, 0))

## Canceling a request

In [ ]:
@resumable function car(sim::Simulation, number::Int, station::Resource)
    while true
        println(nowDatetime(sim), ": ($number) I'm driving")
        @yield timeout(sim, Minute(Int(floor(180*rand()))))
        starttime = nowDatetime(sim)
        println(starttime, ": ($number) I'm parked")
        req = request(station)
        @yield req | timeout(sim, Minute(Int(floor(60*rand()))))
        if state(req) ≠ SimJulia.idle
            stoptime = nowDatetime(sim)
            println(stoptime, ": ($number) I can charge after $(floor(stoptime - starttime, Minute)) minutes")
            @yield timeout(sim, Minute(Int(floor(120*rand()))))
            println(nowDatetime(sim), ": ($number) I'm charged'")
            @yield release(station)
        else
            println(nowDatetime(sim), ": ($number) I'm not charged'")
            cancel(station, req)
        end
    end
end     

In [ ]:
sim = Simulation(DateTime(2018, 10, 23, 8, 45, 0))
charging_station = Resource(sim, 2)
@process  cargenerator(sim, 8, charging_station)
run(sim, DateTime(2018, 10, 23, 24, 0, 0))

## Container

In [ ]:
container = Container(sim, 100.0; level=10.0)
put(container, 20.0)
get(container, 30.0)

## Store

In [ ]:
store = Store{String}(sim; capacity=UInt64(10))
put(store, "Ben Lauwens")
get(store, v->v=="Ben Lauwens")

# Router

In [ ]:
using Pkg
pkg"add Distributions"
pkg"add HypothesisTests"

In [ ]:
using Distributions
using HypothesisTests
using Plots

In [ ]:
const λ = 0.5
const μ = 1.0

In [ ]:
struct SimulationResults
    arrival_times :: Vector{Float64}
    departure_times :: Vector{Float64}
    service_times :: Vector{Float64}
    SimulationResults() = new(Float64[], Float64[], Float64[])
end

In [ ]:
@resumable function packet_process(sim::Simulation, router::Resource, service::Distribution, results::SimulationResults)
    #println(now(sim), ": I arrived!")
    t = now(sim)
    push!(results.arrival_times, now(sim))
    @yield request(router)
    #println(now(sim), ": Processing starts!")
    push!(results.service_times, rand(service))
    @yield timeout(sim, results.service_times[end])
    #println(now(sim), ": I am served!")
    @yield release(router)
    push!(results.departure_times, now(sim))
    #println("I was ", now(sim)-t, " in the router!")
end

In [ ]:
@resumable function source_process(sim::Simulation, results::SimulationResults)
    router = Resource(sim)
    arrival_dist = Exponential(1/λ)
    service_dist = Exponential(1/μ)
    while true
        @process packet_process(sim, router, service_dist, results)
        @yield timeout(sim, rand(arrival_dist))
    end
end

In [ ]:
sim = Simulation()
results = SimulationResults()
@process source_process(sim, results)
run(sim, 1000)

In [ ]:
n = length(results.departure_times)
plot(results.arrival_times[1:n], 1:n, line=:steppre)
plot!(results.departure_times[1:n], 1:n, line=:steppre)

In [ ]:
wait_dist = Exponential(1/(μ-λ))
plot(sort(results.departure_times -  results.arrival_times[1:n]),0:1/(n-1):1)
plot!(0:0.1:10, cdf.(wait_dist, 0:0.1:10))

In [ ]:
W = mean(results.departure_times -  results.arrival_times[1:n])
println(W, " ", mean(wait_dist))

In [ ]:
waiting_times = results.departure_times -  results.arrival_times[1:n]
mean_waiting_times = cumsum(waiting_times)./(1:n)
plot(mean_waiting_times)

In [ ]:
ExactOneSampleKSTest(results.departure_times -  results.arrival_times[1:n], wait_dist)